<a href="https://colab.research.google.com/github/Uditsingh7/Rag-Projects/blob/main/Udit_Rag_Project_GitHub_Issues.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Building** a RAG using an open-source LLM, embeddings model, and LangChain.

In [ ]:
## install dependencies

!pip install -q torch transformers accelerate bitsandbytes sentence-transformers

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!pip install faiss-gpu-cu12

In [ ]:
## q=quiet
!pip install -q langchain

In [ ]:
!pip install -q langchain_community

In [ ]:
!pip install -q sentence-transformers

In [ ]:
from getpass import getpass

Access_Token = getpass("Please enter your GitHub Personal Access Token: ")


Please enter your GitHub Personal Access Token: ··········


In [ ]:
from os import access
##  load all of the issues in the huggingface/peft repo

from langchain.document_loaders import GitHubIssuesLoader

## By default, pull requests are considered issues as well,
##here we chose to exclude them from data with by setting include_prs=False

## state=all, means we will load both open and closed isses

repo = "huggingface/peft"
loader = GitHubIssuesLoader(repo=repo, access_token=Access_Token,
                            include_prs=False, state="all")

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

Document(metadata={'url': 'https://github.com/huggingface/peft/issues/2566', 'title': 'Found missing adapter keys while loading the checkpoint', 'creator': 'JJplane', 'created_at': '2025-06-03T09:56:08Z', 'comments': 3, 'state': 'open', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 2566, 'is_pull_request': False}, page_content='```\nself.model = MllamaForConditionalGeneration.from_pretrained(\n            self.model_name,\n            device_map="auto",\n            torch_dtype=torch.float16\n        )\nself.model = PeftModel.from_pretrained(self.model, self.lora_path)\n```\n\n/root/miniconda3/envs/crag/lib/python3.10/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: [\'base_model.model.model.vision_model.transformer.layers.0.self_attn.q_proj.lora_A.default.weight\', \'base_model.model.model.vision_model.transformer.layers.0.self_attn.q_proj.lora_B.default.weight\', \'base_model.model.model.vision

In [ ]:
## Keeping some overlap between chunks allows us to preserve some semantic context between the chunks.

## The recommended splitter for generic text is the RecursiveCharacterTextSplitter

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=30,
)

chnked_docs = splitter.split_documents(docs)

chnked_docs[0]

Document(metadata={'url': 'https://github.com/huggingface/peft/issues/2566', 'title': 'Found missing adapter keys while loading the checkpoint', 'creator': 'JJplane', 'created_at': '2025-06-03T09:56:08Z', 'comments': 3, 'state': 'open', 'labels': [], 'assignee': None, 'milestone': None, 'locked': False, 'number': 2566, 'is_pull_request': False}, page_content='```\nself.model = MllamaForConditionalGeneration.from_pretrained(\n            self.model_name,\n            device_map="auto",\n            torch_dtype=torch.float16\n        )\nself.model = PeftModel.from_pretrained(self.model, self.lora_path)\n```')

In [ ]:
## Created embedders and Vectors

## To create document chunk embeddings we’ll use theHuggingFaceEmbeddings and the BAAI/bge-base-en-v1.5 embeddings model

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

embeddings

<ipython-input-12-1023b01e0a8c>:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-base-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [ ]:
## To create the vector database, we’ll use FAISS, a library developed by Facebook AI.
##This library offers efficient similarity search and clustering of dense vectors, which is what we need here.
##FAISS is currently one of the most used libraries for NN search in massive datasets.

from langchain.vectorstores import FAISS

db = FAISS.from_documents(chnked_docs, embeddings)

db


In [ ]:
## We need a way to return(retrieve) the documents given an unstructured query.
##For that, we’ll use the as_retriever method using the db as a backbone

## Declare a retriever method with the vector db
retriever = db.as_retriever(search_type='similarity', search_kwargs= {"k":4})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79a0788aa790>, search_kwargs={'k': 4})

In [ ]:
## The vector database and retriever are now set up,
##next we need to set up the next piece of the chain — the model.


In [ ]:
## Load quantized model
##  we chose HuggingFaceH4/zephyr-7b-beta, a small but powerful model.

## To make inference faster, will load the quantized version of the model:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
 

In [ ]:
## We have all the pieces we need to set the LLM chain
##loaded doc, splitted the doc, chnked it, embedded it, stored it in the vector db,
##defined a retriever method, loaded the qantised model, tokeniser


## Setup the LLM chain

## first create a text generation pipeline  using the loaded model and its tokenizer.

from transformers import pipeline
from langchain.llms import HuggingFacePipeline

text_generation_pipeline = pipeline(
    model=model, # The pre-trained language model object itself. This is typically an instance of a Hugging Face 'AutoModelForCausalLM'.
    tokenizer=tokenizer, # The tokenizer corresponding to the 'model'. It's essential for converting text to token IDs and vice-versa, which the model understands.
    task='text-generation', # Specifies the task for the pipeline. 'text-generation' means the pipeline will generate new text based on a given prompt.
    temperature=0.2, # Controls the randomness of the generated text. A value of 0.2 (low) makes the output more deterministic and focused on high-probability tokens, reducing creativity.
    do_sample=True, # Enables sampling-based text generation. When True, the model will pick tokens stochastically based on their probabilities, rather than just the most probable one.
    repetition_penalty=1.1, # Penalizes the generation of repeated tokens. A value of 1.1 (slightly above 1.0) discourages the model from repeating words or phrases too often, improving coherence.
    return_full_text=True, # Determines if the output should include the input prompt along with the generated text. True means the full text (prompt + generation) is returned.
    max_new_tokens=512, # The maximum number of new tokens to generate in addition to the input prompt. A value of 512 sets a reasonable limit on the length of the generated output.
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
llm


Device set to use cuda:0
<ipython-input-21-b57174180868>:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x799f933fbcd0>)

In [ ]:
## Next, create a prompt template — this should follow the format of the model, so if you substitute the model checkpoint,
##make sure to use the appropriate formatting.

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

## StrOutputParser: take the output from a Language Model (LLM) and ensure it's returned as a plain string.
##Basically extration of content data in the form os string

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:
{context}
</s>
<|user|>
{question}
</s>
<|assistant|>
 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template
)

## form a llm chain with prompt and llm
## | stands for LangChain Expression Language (LCEL).
##Its is sed to form a pipe or sequential composition
### It means o/p of starting from left will be  automatically passed on to the next as an i/p
llm_chain = prompt | llm | StrOutputParser()

llm_chain





PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n{context}\n</s>\n<|user|>\n{question}\n</s>\n<|assistant|>\n ')
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x799f933fbcd0>)
| StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

rag_chain

{
  context: VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x79a0788aa790>, search_kwargs={'k': 4}),
  question: RunnablePassthrough()
}
| PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n{context}\n</s>\n<|user|>\n{question}\n</s>\n<|assistant|>\n')
| HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x799f933fbcd0>)
| StrOutputParser()

In [ ]:
question = "How do you combine multiple adapters?"

In [ ]:
## First, let’s see what kind of answer we can get with just the model itself, no context added

llm_chain.invoke({"context":"", "question":question})

"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n</s>\n<|user|>\nHow do you combine multiple adapters?\n</s>\n<|assistant|>\n  To combine multiple adapters, you need to ensure that they are compatible with each other and the devices you want to connect. Here's how you can do it:\n\n1. Identify the adapters you need: Determine which adapters you require to connect your devices. For example, if you want to connect a USB-C device to an HDMI monitor, you may need a USB-C to HDMI adapter as well as a USB-C to USB-A adapter.\n\n2. Connect the first adapter: Plug in the first adapter into the device you want to connect. For instance, if you're connecting a USB-C laptop to an HDMI monitor, insert the USB-C to USB-A adapter into the laptop's USB-C port.\n\n3. Attach the second adapter: Next, attach the second adapter to the first one. In this case, connect the USB-A end of the USB-C to USB-A adapter to the USB-C to HDMI adapter.\n\n4. Connect the

In [ ]:
rag_chain.invoke(question)

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n[Document(id=\'fff02d8b-14a6-49f8-a38d-482d1bb8fbf5\', metadata={\'url\': \'https://github.com/huggingface/peft/issues/1802\', \'title\': \'Issues when switching between multiple adapters LoRAs \', \'creator\': \'JhonDan1999\', \'created_at\': \'2024-05-26T19:18:13Z\', \'comments\': 8, \'state\': \'closed\', \'labels\': [], \'assignee\': None, \'milestone\': None, \'locked\': False, \'number\': 1802, \'is_pull_request\': False}, page_content=\'The documentation does not mention the need to perform a merge when switching adapters. Additionally, the methods add_adapter, set_adapter, and enable_adapters do not appear to work\\r\\n\\r\\nPlease provide clarification on how to correctly switch between adapters\'), Document(id=\'fb30c3e9-53ee-4909-8043-86256f8a7adb\', metadata={\'url\': \'https://github.com/huggingface/peft/issues/1045\', \'title\': \'add_weighted_adapter() is unusable, throws erro